In [1]:
## package setup and import
import numpy as np
import datajoint as dj
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from pathlib import Path
from scipy.io import loadmat

In [2]:
## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [3]:
## SETTING UP DATA STORES

datastorefolder = 'D:\Dropbox\BpodInfoseek\Data\''

dj.config['stores'] = {
    'raw_data':{
        'protocol':'file',
        'location': os.path.join(datastorefolder,'raw_data')
    }
}

dj.config.save_global()

In [4]:
## ESTABLISH SCHEMA

schema = dj.schema('simple')
schema.drop(True)
schema = dj.schema('simple')

In [5]:
## Manual entry table for mouse identification/attribute labelling
@schema
class Mice(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : varchar(100) 
    ---
    sex = 'U': enum('F','M','U')
    
    geno = 'C57BL6' : varchar(50)
    """

#     DOB = null : date  # YYYY-MM-DD
#     imaging = null : int

# format for entry of new mouse with new attributes, load later on will create new mouse if not previously entered but will be missing experimental params
mice = Mice()

mice.insert([{'mouse_id' : 'JB366', 'sex' : 'M'},{'mouse_id' : 'JB367','sex' :'F'},{'mouse_id' : 'JB368', 'sex' : 'M'},{'mouse_id' : 'JB369', 'sex' : 'M'}],skip_duplicates=True)
mice

# dj.ERD(schema)

mouse_id,sex,geno
JB366,M,C57BL6
JB367,F,C57BL6
JB368,M,C57BL6
JB369,M,C57BL6


In [6]:
mice.proj('sex')

mouse_id,sex
JB366,M
JB367,F
JB368,M
JB369,M


In [7]:
# List all data files in directory using pathlib for insertion in manual table

files = []
# datafolder = Path('D:\Dropbox\BpodInfoseek\Data\PythonTest')
datafolder = Path(r'C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest')
for file in datafolder.glob('JB*.mat'):
    if file.stat().st_size>3000: # leaves out garbage files!
        thisfile = {}
        thisfile["filename"] = file.name
        thisfile["filepath"] = str(file.parent)
        mouseID,protocol,datestr,timestr = file.stem.split('_')
        thisfile["mouse_id"] = mouseID
        thisfile["protocol"] = protocol
        date_obj = datetime.datetime.strptime(datestr, '%Y%m%d')
        thisfile["date"] = date_obj.date()
        time_obj = datetime.datetime.strptime(timestr, '%H%M%S')
#         time_obj = datetime.datetime.strptime(timestr[:len(timestr)-4], '%H%M%S')
        thisfile["time"] = time_obj.time()
        files.append(thisfile)

In [8]:
filesDf = pd.DataFrame(files)
# filesDf.groupby(['mouse_id','date']).sum()
# columns = filesDf[["mouse_id","date"]].values
# mousedays = pd.unique(columns)
# print(mousedays)
mousedaysDf = filesDf.groupby(['mouse_id','date']).size().reset_index().rename(columns={0:'sessions'})
mousedaysDfshort = mousedaysDf[['mouse_id','date']]
# mousedaysDfshort

In [9]:
@schema
class MouseDays(dj.Manual):
    definition = """
    ->Mice
    date : date
    ---
    """
mouseDays = MouseDays()
mouseDays.insert(mousedaysDfshort)
mouseDays

mouse_id,date
JB366,2020-10-13
JB366,2020-10-14
JB366,2020-10-15
JB366,2020-10-16
JB367,2020-10-13
JB367,2020-10-14
JB367,2020-10-15
JB367,2020-10-16
JB368,2020-10-12
JB368,2020-10-13


In [10]:
# BehaviorSession.drop(True)
@schema
class BehaviorSession(dj.Manual):
    definition = """
    # Infoseek behavior .mat files from a folder
    filename : varchar(127)
    -> MouseDays
    time : time
    protocol : varchar(127)      
    ---  
    filepath : varchar(127)
    """
    


In [11]:
behaviorSession = BehaviorSession()
behaviorSession.insert(files,skip_duplicates=True)
behaviorSession

filename,mouse_id,date,time,protocol,filepath
JB366_Infoseek_20201013_102926.mat,JB366,2020-10-13,10:29:26,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201014_103353.mat,JB366,2020-10-14,10:33:53,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201014_104211.mat,JB366,2020-10-14,10:42:11,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201015_101140.mat,JB366,2020-10-15,10:11:40,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201016_102403.mat,JB366,2020-10-16,10:24:03,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201016_104533.mat,JB366,2020-10-16,10:45:33,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201013_112258.mat,JB367,2020-10-13,11:22:58,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201014_112858.mat,JB367,2020-10-14,11:28:58,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201015_105831.mat,JB367,2020-10-15,10:58:31,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201015_111113.mat,JB367,2020-10-15,11:11:13,Infoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest


In [12]:
filename = r'JB366_TestInfoseek_20200916_113754.mat'
filepath = r'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,filename)
print(newpath)

D:\Dropbox\BpodInfoseek\Data\PythonTest\JB366_TestInfoseek_20200916_113754.mat


In [23]:
name = 'JB366_TestInfoseek_20200916_113754.mat'
'r'+name
filepath = 'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,name)
print(newpath)

D:\Dropbox\BpodInfoseek\Data\PythonTest\JB366_TestInfoseek_20200916_113754.mat


In [28]:
# SessionRawData.drop(True)

# Save raw data

@schema
class SessionRawData(dj.Imported):
    definition = """
    -> BehaviorSession
    ---
    raw_data   : longblob     # raw 'session_data' from Bpod matlab file
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        name = (BehaviorSession & key).fetch1()['filename']
        partpath = (BehaviorSession & key).fetch1()['filepath']
        matdata = loadmat(Path(partpath,name))
        key['raw_data'] = pd.DataFrame(matdata['SessionData'][0])
        self.insert1(key)

In [30]:
SessionRawData.populate()
SessionRawData()

filename,mouse_id,date,raw_data raw 'session_data' from Bpod matlab file
JB366_Infoseek_20201013_102926.mat,JB366,2020-10-13,=BLOB=
JB366_Infoseek_20201014_103353.mat,JB366,2020-10-14,=BLOB=
JB366_Infoseek_20201014_104211.mat,JB366,2020-10-14,=BLOB=
JB366_Infoseek_20201015_101140.mat,JB366,2020-10-15,=BLOB=
JB366_Infoseek_20201016_102403.mat,JB366,2020-10-16,=BLOB=
JB366_Infoseek_20201016_104533.mat,JB366,2020-10-16,=BLOB=
JB367_Infoseek_20201013_112258.mat,JB367,2020-10-13,=BLOB=
JB367_Infoseek_20201014_112858.mat,JB367,2020-10-14,=BLOB=
JB367_Infoseek_20201015_105831.mat,JB367,2020-10-15,=BLOB=
JB367_Infoseek_20201015_111113.mat,JB367,2020-10-15,=BLOB=
